# Exploring Glider Data with Jupyter Notebook and IOOS Map

**Useful links**
- [IOOS](https://gliders.ioos.us/)
- [IOOS map](https://gliders.ioos.us/map/)
- [ERDDAP OOI Demo](https://www.youtube.com/watch?v=tj4M9hodTG0)
- [reproducible-notebooks - ERDDAP_glider_search](https://github.com/reproducible-notebooks/ERDDAP_glider_search)

## Read glider data from ERDDAP

In [ ]:
from erddapy import ERDDAP
import pandas as pd

def get_erddap_dataset(ds_id, server, variables=None, constraints=None, filetype=None):
    #  Available at: 
    #      https://github.com/JGradone/Slocum-AD2CP/blob/main/src/analysis/analysis.py
    """
    Fetch a dataset from an ERDDAP server.
    Parameters:
        - ds_id: dataset ID e.g. ng314-20200806T2040
        - variables: optional list of variables
        - constraints: optional list of constraints
        - filetype: optional filetype to return, 'nc' (default) or 'dataframe'
    Returns:
        - xarray.Dataset: If filetype is 'nc', returns an xarray Dataset object.
        - pandas.DataFrame: If filetype is 'dataframe', returns a pandas DataFrame.
          If the dataset cannot be converted to xarray, it also returns a DataFrame.
    """
    variables = variables or None
    constraints = constraints or None
    filetype = filetype or 'nc'


    e = ERDDAP(server,
               protocol='tabledap',
               response='nc')
    e.dataset_id = ds_id
    if constraints:
        e.constraints = constraints
    if variables:
        e.variables = variables
    if filetype == 'nc':
        try:
            ds = e.to_xarray()
            ds = ds.sortby(ds.time)
        except OSError:
            print('No dataset available for specified constraints: {}'.format(ds_id))
            ds = []
        except TypeError:
            print('Cannot convert to xarray, providing dataframe: {}'.format(ds_id))
            ds = e.to_pandas().dropna()
    elif filetype == 'dataframe':
        ds = e.to_pandas().dropna(how='all')
    else:
        print('Unrecognized filetype: {}. Needs to  be "nc" or "dataframe"'.format(filetype))

    return ds

**Load glider data directly from ERDDAP as xarray**

The test dataset can be found at the following link:
- [test data](https://gliders.ioos.us/erddap/search/index.html?page=1&itemsPerPage=1000&searchFor=ng296-20230726T172700)

After applying time constraints on the test dataset, some plots can be found at the following links:
- [temperature](https://gliders.ioos.us/erddap/tabledap/ng296-20230726T172700.graph?time%2Cdepth%2Ctemperature&time%3E=2023-12-13T00%3A00%3A00Z&time%3C=2023-12-15T00%3A00%3A00Z&.draw=markers&.marker=5%7C5&.color=0xFFFFFF&.colorBar=%7C%7C%7C%7C%7C&.bgColor=0xffccccff)
- [location](https://gliders.ioos.us/erddap/tabledap/ng296-20230726T172700.graph?latitude%2Clongitude%2Cprofile_id&time%3E=2023-12-13T00%3A00%3A00Z&time%3C=2023-12-15T00%3A00%3A00Z&.draw=markers&.marker=5%7C5&.color=0xFFFFFF&.colorBar=%7C%7C%7C%7C%7C&.bgColor=0xffccccff)

In [ ]:
## Load data

ds_id = 'ng296-20230726T172700'
min_time = '2023-12-13T00:00:00Z'
max_time = '2023-12-15T00:00:00Z'
constraints = {
    'time>=': min_time,
    'time<=': max_time,
}
variables = ['depth', 'latitude', 'longitude', 'time', 'temperature']
gdf = get_erddap_dataset(
    ds_id, server='https://gliders.ioos.us/erddap', 
    variables = variables, 
    constraints=constraints, 
    filetype='nc')
gdf

**Different results in data visualization:**

Scenario 1: Utilizing the get_erddap_dataset function with filetype='nc' results in an xarray.Dataset. Converting this dataset to a Pandas DataFrame and creating a plot (for instance, time versus depth with temperature as color) reveals that the plot differs from [the predefined reference plot](https://gliders.ioos.us/erddap/tabledap/ng296-20230726T172700.graph?time%2Cdepth%2Ctemperature&time%3E=2023-12-13T00%3A00%3A00Z&time%3C=2023-12-15T00%3A00%3A00Z&.draw=markers&.marker=5%7C5&.color=0xFFFFFF&.colorBar=%7C%7C%7C%7C%7C&.bgColor=0xffccccff).

Scenario 2: Alternatively, data can be downloaded locally from the ERDDAP server as a netCDF file followed by reading it into Python using xr.open_dataset(file_path, engine='netcdf4', decode_times=False) and then converting this dataset to a Pandas DataFrame, yields a plot that aligns with [the expected reference](https://gliders.ioos.us/erddap/tabledap/ng296-20230726T172700.graph?time%2Cdepth%2Ctemperature&time%3E=2023-12-13T00%3A00%3A00Z&time%3C=2023-12-15T00%3A00%3A00Z&.draw=markers&.marker=5%7C5&.color=0xFFFFFF&.colorBar=%7C%7C%7C%7C%7C&.bgColor=0xffccccff).

In [ ]:
# Scenario 1:

import hvplot.pandas

gdf = gdf.to_dataframe().reset_index()
gdf['time'] = pd.to_datetime(gdf['time'], unit='s')
gdf.hvplot.scatter(
    x='time', 
    y='depth', 
    c='temperature', 
    cmap='viridis', 
    size=8, 
    height=500, 
    width=700
).opts(
    invert_yaxis=True,
    colorbar=True,
    colorbar_opts={'title': 'Temperature (°C)'}
)


In [ ]:
import plotly.graph_objects as go

optimized_lon_lat = gdf[['longitude', 'latitude']].drop_duplicates()

fig = go.Figure(data=go.Scattergeo(
    lon = optimized_lon_lat['longitude'],
    lat = optimized_lon_lat['latitude'],
    mode = 'lines+markers',
    line = dict(width = 2, color = 'blue'),
    marker = dict(size = 3, color = 'red', opacity = 0.7) 
))


fig.update_layout(
    title_text = 'Glider Path with Map Background',
    showlegend = False,
    geo = dict(
        scope = 'world',
        projection_type = 'equirectangular',
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
        resolution = 50,
        center = dict(lon = optimized_lon_lat['longitude'].mean(), lat = optimized_lon_lat['latitude'].mean()),
        projection_scale = 5 
    ),
)

fig.show()

In [ ]:
import plotly.express as px


fig = px.scatter(gdf, x='longitude', y='latitude')


fig.update_layout(
    title='Scatter Plot for ng296-20230726T172700',
    xaxis_title='Longitude [degrees_east]',
    yaxis_title='Latitude [degrees_north]',
    yaxis = dict(
        scaleanchor = "x",
        scaleratio = 1,
      ),
)

# Show the plot
fig.show()

**Load glider data using a downloaded netCDF file from ERDDAP**

In [ ]:
# Scenario 2:

import os
import hvplot.pandas
import xarray as xr


current_directory = os.getcwd()
file_path = os.path.join(current_directory,
                         'assets',
                         'glider_data',
                         'ng296-20230726T172700_8afc_ce38_9a62.nc')

ds = xr.open_dataset(file_path, engine='netcdf4', decode_times=False)

ds = ds.to_dataframe().reset_index()

ds['time'] = pd.to_datetime(ds['time'], unit='s')

ds.hvplot.scatter(
    x='time', 
    y='depth', 
    c='temperature', 
    cmap='viridis', 
    size=8, 
    height=500, 
    width=700
).opts(
    invert_yaxis=True,
    colorbar=True,
    colorbar_opts={'title': 'Temperature (°C)'}
)

In [ ]:
import plotly.graph_objects as go

optimized_lon_lat = gdf[['longitude', 'latitude']].drop_duplicates()

fig = go.Figure(data=go.Scattergeo(
    lon = optimized_lon_lat['longitude'],
    lat = optimized_lon_lat['latitude'],
    mode = 'lines+markers',
    line = dict(width = 2, color = 'blue'),
    marker = dict(size = 3, color = 'red', opacity = 0.7) 
))


fig.update_layout(
    title_text = 'Glider Path with Map Background',
    showlegend = False,
    geo = dict(
        scope = 'world',
        projection_type = 'equirectangular',
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
        resolution = 50,
        center = dict(lon = optimized_lon_lat['longitude'].mean(), lat = optimized_lon_lat['latitude'].mean()),
        projection_scale = 5 
    ),
)

fig.show()

In [ ]:
import plotly.express as px


fig = px.scatter(gdf, x='longitude', y='latitude')


fig.update_layout(
    title='Scatter Plot for ng296-20230726T172700',
    xaxis_title='Longitude [degrees_east]',
    yaxis_title='Latitude [degrees_north]',
    yaxis = dict(
        scaleanchor = "x",
        scaleratio = 1,
      ),
)

# Show the plot
fig.show()

**Load glider data directly from ERDDAP as Pandas DataFrame**

The test dataset can be found at the following link:
- [test data](https://gliders.ioos.us/erddap/search/index.html?page=1&itemsPerPage=1000&searchFor=ng296-20230726T172700)

After applying time constraints on the test dataset, some plots can be found at the following links:
- [temperature](https://gliders.ioos.us/erddap/tabledap/ng296-20230726T172700.graph?time%2Cdepth%2Ctemperature&time%3E=2023-12-13T00%3A00%3A00Z&time%3C=2023-12-15T00%3A00%3A00Z&.draw=markers&.marker=5%7C5&.color=0xFFFFFF&.colorBar=%7C%7C%7C%7C%7C&.bgColor=0xffccccff)
- [location](https://gliders.ioos.us/erddap/tabledap/ng296-20230726T172700.graph?latitude%2Clongitude%2Cprofile_id&time%3E=2023-12-13T00%3A00%3A00Z&time%3C=2023-12-15T00%3A00%3A00Z&.draw=markers&.marker=5%7C5&.color=0xFFFFFF&.colorBar=%7C%7C%7C%7C%7C&.bgColor=0xffccccff)

In [ ]:
## Load data
# The dataset can be found at the following link:
    # https://gliders.ioos.us/erddap/search/index.html?page=1&itemsPerPage=1000&searchFor=ng296-20230726T172700
ds_id = 'ng296-20230726T172700'
min_time = '2023-12-13T00:00:00Z'
max_time = '2023-12-15T00:00:00Z'
constraints = {
    'time>=': min_time,
    'time<=': max_time,
}
variables = ['depth', 'latitude', 'longitude', 'time', 'temperature']
gdf = get_erddap_dataset(
    ds_id, server='https://gliders.ioos.us/erddap', 
    variables = variables, 
    constraints=constraints, 
    filetype='dataframe')
gdf

In [ ]:
gdf.columns = variables
# Convert 'time (UTC)' to datetime and set as index
gdf['time'] = pd.to_datetime(gdf['time'])
gdf = gdf.set_index('time')

In [ ]:
gdf

In [ ]:
number_of_unique_timestamps = gdf.index.nunique()

print(f'There are {number_of_unique_timestamps} unique timestamps in the DataFrame.')

**In oceanographic data collection, high-resolution sensors on gliders often record multiple variables at the same timestamp, a challenge for formats like netCDF which typically map single observations to discrete time points, particularly in its default format used in ERDDAP. Pandas DataFrames adeptly handle this by allowing duplicate timestamps, supporting irregular time series, and providing multi-level indexing. This ensures that detailed sensor readings are accurately maintained and aligned for analysis, showcasing Pandas' capability to manage complex environmental datasets.**

- In the context of the example dataset from ERDDAP, read into both Pandas and netCDF formats, Pandas DataFrames uniquely manage temporal data by retaining all timestamps, including duplicates. When assessing unique time values in this dataset, Pandas identifies 35 distinct timestamps, mirroring the netCDF format's categorization under xarray, which segregates 35 unique profile points against 15601 observations. This disparity arises as Pandas directly maps each observation to its respective timestamp, regardless of duplicates, offering a straightforward approach for detailed time-series analysis. In contrast, netCDF/xarray's separation of time (profile) and observations (obs) into distinct dimensions introduces complexities in precisely aligning each data point with its corresponding time instance, particularly in datasets with high-frequency measurements recorded at single time points.

In [ ]:
import plotly.graph_objs as go

optimized_lon_lat = gdf[['longitude', 'latitude']].drop_duplicates()

fig = go.Figure(data=go.Scattergeo(
    lon = optimized_lon_lat['longitude'],
    lat = optimized_lon_lat['latitude'],
    mode = 'lines+markers',
    line = dict(width = 2, color = 'blue'),
    marker = dict(size = 3, color = 'red', opacity = 0.7) 
))


fig.update_layout(
    title_text = 'Glider Path with Map Background',
    showlegend = False,
    geo = dict(
        scope = 'world',
        projection_type = 'equirectangular',
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
        resolution = 50,
        center = dict(lon = optimized_lon_lat['longitude'].mean(), lat = optimized_lon_lat['latitude'].mean()),
        projection_scale = 5 
    ),
)

fig.show()


In [ ]:
import plotly.express as px


fig = px.scatter(gdf, x='longitude', y='latitude')


fig.update_layout(
    title='Scatter Plot for ng296-20230726T172700',
    xaxis_title='Longitude [degrees_east]',
    yaxis_title='Latitude [degrees_north]',
    yaxis = dict(
        scaleanchor = "x",
        scaleratio = 1,
      ),
)

# Show the plot
fig.show()

In [ ]:
import hvplot.pandas

gdf.hvplot.scatter(
    x='time', 
    y='depth', 
    c='temperature', 
    cmap='viridis', 
    size=8, 
    height=500, 
    width=700
).opts(
    invert_yaxis=True,
    colorbar=True,
    colorbar_opts={'title': 'Temperature (°C)'}
)

## Read glider data downloaded from THREDDS as an ‘nc’ file

- The test data can be found at the following [link](https://gliders.ioos.us/thredds/catalog/deployments/mbari/UW646-20220907T0000/catalog.html?dataset=deployments/mbari/UW646-20220907T0000/UW646-20220907T0000.nc3.nc)

In [ ]:
import os
import xarray as xr


current_directory = os.getcwd()
file_path = os.path.join(current_directory, 'assets', 'glider_data', 'UW646-20220907T0000.nc3.nc')

# Open the NetCDF file
dataset = xr.open_dataset(file_path)

# Explore the dataset
dataset


In [ ]:
import hvplot.pandas

# Open the NetCDF file - using engine='netcdf4', decode_times=False
ds = xr.open_dataset(file_path, engine='netcdf4', decode_times=False)

ds = ds.to_dataframe().reset_index()

ds['time'] = pd.to_datetime(ds['time'], unit='s')

ds.hvplot.scatter(
    x='time', 
    y='depth', 
    c='temperature', 
    cmap='plasma', 
    size=8, 
    height=500, 
    width=700
).opts(
    invert_yaxis=True,
    colorbar=True,
    colorbar_opts={'title': 'Temperature (°C)'}
)

## Read glider data  downloaded from ERDDAP as CSV file

**First, we conduct some tests using the ERDDAP server directly for this dataset:** [HERE](https://gliders.ioos.us/erddap/tabledap/sg625-20230919T0000.html)

- Data with time constraints

In [ ]:
variables = [
    'depth',
    'latitude',
    'longitude',
    'salinity',
    'temperature',
    'density',
    'conductivity',
    'time',
]

ds_id = 'sg625-20230919T0000'

server = 'https://gliders.ioos.us/erddap'

min_time = '2023-11-28T00:00:00Z'
max_time = '2023-12-05T14:34:20Z'
constraints = {
    'time>=': min_time,
    'time<=': max_time,
}


gdf = get_erddap_dataset(ds_id, server=server, variables=variables, constraints=constraints, filetype='dataframe')

gdf.columns = variables


gdf


In [ ]:
print(gdf.dtypes)

When visualizing oceanographic data collected over time, such as from gliders or buoys, it is common to encounter duplicate timestamps in the dataset, as with our example DataFrame. These duplications can occur due to various reasons, like multiple sensors recording different parameters at the same time.
In the provided `plot_depth_profile` function, the handling of potential duplicates is approached systematically:

1. **DataFrame Copy**: The function begins by creating a copy of the input DataFrame. This ensures that any modifications done during the plotting process do not alter the original data.

2. **Datetime Conversion**: The `time` column is converted to datetime objects using `pd.to_datetime`. This standardizes the time information, ensuring that all time-related operations are performed accurately.

3. **Indexing by Time**: By setting the `time` column as the index of the DataFrame, we facilitate time-based indexing and plotting. This is important for time series analysis and visualization.

4. **Data Cleaning**: Before plotting, rows with NaN values in either the variable of interest or the 'depth' column are removed using `df_cleaned = df_copy.dropna(subset=[variable, 'depth'])`. This step ensures that only valid data points are included in the plot.

5. **Scatter Plot**: A scatter plot is created where each point represents a measurement. When there are multiple measurements for the same timestamp (i.e., duplicates), they are all plotted. This could mean that for any given time, there could be several depth readings, each associated with a value for the specified variable. The color of each point reflects the magnitude of the variable.

6. **Color Coding**: The variable of interest is represented by the color of the points in the scatter plot, with the color intensity corresponding to the variable's value. This provides a visual representation of changes in the variable with respect to both depth and time.

7. **Axis Formatting**: The y-axis is inverted (`ax.invert_yaxis()`) to align with the oceanographic convention of displaying depth, where deeper depths are shown lower on the plot. The x-axis is formatted to display the date and time clearly.

By implementing these steps, the function `plot_depth_profile` creates a clear and informative visualization of the measured variable over time and depth, even in the presence of duplicates. The resulting plots allow for the examination of temporal changes in oceanographic parameters across different depths.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd


def plot_depth_profile(df, variable, color_label, colormap):
    # Work on a copy to not modify the original DataFrame
    df_copy = df.copy()
    
    # Convert the 'time' column to datetime and set as index
    df_copy['time'] = pd.to_datetime(df_copy['time'])
    df_copy.set_index('time', inplace=True)
    
    # Drop rows where the variable or 'depth' is NaN
    df_cleaned = df_copy.dropna(subset=[variable, 'depth'])
    
    # Creating the scatter plot
    fig, ax = plt.subplots(figsize=(17, 2))
    scatter = ax.scatter(df_cleaned.index, df_cleaned['depth'], s=15, c=df_cleaned[variable], cmap=colormap, marker='o', edgecolor='none')
    
    # Inverting the y-axis for depth
    ax.invert_yaxis()
    
    # Formatting the x-axis to show time
    xfmt = mdates.DateFormatter('%H:%Mh\n%d-%b')
    ax.xaxis.set_major_formatter(xfmt)
    
    # Adding colorbar and labels
    cbar = fig.colorbar(scatter, orientation='vertical', extend='both')
    cbar.ax.set_ylabel(color_label)
    ax.set_ylabel('Depth (m)')
    ax.set_xlabel('Time')
    
    plt.show()


In [ ]:
plot_depth_profile(gdf, 'temperature', 'Temperature (°C)', 'plasma')
plot_depth_profile(gdf, 'salinity', 'Salinity (1e-3)', 'viridis')
plot_depth_profile(gdf, 'conductivity', 'Conductivity (S m$^{-1}$)', 'viridis')
plot_depth_profile(gdf, 'density', 'Density (kg m$^{-3}$)', 'PuBu')

- Data without time constraints

In [ ]:
variables = [
    'depth',
    'latitude',
    'longitude',
    'salinity',
    'temperature',
    'density',
    'conductivity',
    'time',
]

ds_id = 'sg625-20230919T0000'

server = 'https://gliders.ioos.us/erddap'


gdf = get_erddap_dataset(ds_id, server=server, variables=variables, filetype='dataframe')

gdf.columns = variables


gdf

plot_depth_profile(gdf, 'temperature', 'Temperature (°C)', 'plasma')
plot_depth_profile(gdf, 'salinity', 'Salinity (1e-3)', 'viridis')
plot_depth_profile(gdf, 'conductivity', 'Conductivity (S m$^{-1}$)', 'viridis')
plot_depth_profile(gdf, 'density', 'Density (kg m$^{-3}$)', 'PuBu')

In [ ]:
import plotly.graph_objs as go

optimized_lon_lat = gdf[['longitude', 'latitude']].drop_duplicates()

fig = go.Figure(data=go.Scattergeo(
    lon = optimized_lon_lat['longitude'],
    lat = optimized_lon_lat['latitude'],
    mode = 'lines+markers',
    line = dict(width = 2, color = 'blue'),
    marker = dict(size = 3, color = 'red', opacity = 0.7) 
))


fig.update_layout(
    title_text = 'Glider Path with Map Background',
    showlegend = False,
    geo = dict(
        scope = 'world',
        projection_type = 'equirectangular',
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
        resolution = 50,
        center = dict(lon = optimized_lon_lat['longitude'].mean(), lat = optimized_lon_lat['latitude'].mean()),
        projection_scale = 5 
    ),
)

fig.show()

**After viewing some plots using data taken directly from the ERDDAP server, let's see if downloading the same data as a CSV file results in the same DataFrame**

In [ ]:
variables = [
    'depth',
    'latitude',
    'longitude',
    'salinity',
    'temperature',
    'density',
    'conductivity',
    'time',
]

ds_id = 'sg625-20230919T0000'

server = 'https://gliders.ioos.us/erddap'

min_time = '2023-11-28T00:00:00Z'
max_time = '2023-12-05T14:34:20Z'
constraints = {
    'time>=': min_time,
    'time<=': max_time,
}


gdf = get_erddap_dataset(ds_id, server=server, variables=variables, constraints=constraints, filetype='dataframe')

gdf.columns = variables


gdf

In [ ]:
print(gdf.dtypes)

**To test if the data downloaded as a CSV and the data obtained directly from the server are the same, the CSV should be downloaded on the same day the test is run. This is important as it seems that new data can be added to the server, even from the past. After downloading, it should be added to `csv_file_path` (found in the code below).**
- To download the data, enter [HERE](https://gliders.ioos.us/erddap/tabledap/sg625-20230919T0000.html), select the time constraints and needed variables, and download the resulting selected data in CSV format.

In [ ]:
import os

current_directory = os.getcwd()

csv_file_path = os.path.join(current_directory, 'assets', 'glider_data', 'sg625-20230919T0000_f192_3cce_1140.csv')
df = pd.read_csv(csv_file_path, header=0, skiprows=[1])
df

In [ ]:
print(df.dtypes)

In [ ]:
import pandas as pd

df2=df.copy()
df1=gdf.copy()

# Align the columns of df2 to match the order of df1
df2_aligned = df2[df1.columns]

# Compare the two DataFrames
are_equal = df1.equals(df2_aligned)

print("Are the DataFrames equal (ignoring column order)?", are_equal)


**Finally, we conduct some tests using the data taken from the CSV file**

In [ ]:
plot_depth_profile(df, 'temperature', 'Temperature (°C)', 'plasma')
plot_depth_profile(df, 'salinity', 'Salinity (1e-3)', 'viridis')
plot_depth_profile(df, 'conductivity', 'Conductivity (S m$^{-1}$)', 'viridis')
plot_depth_profile(df, 'density', 'Density (kg m$^{-3}$)', 'PuBu')